In [116]:
import random
from deap import base, creator, tools, algorithms
import numpy as np

In [117]:
# Define constants for traffic light timings
GREEN_MIN, GREEN_MAX = 10, 60   # Green light duration (seconds)
YELLOW_MIN, YELLOW_MAX = 3, 10   # Yellow light duration (seconds)
RED_MIN, RED_MAX = 10, 60       # Red light duration (seconds)

In [118]:
# Define traffic parameters (example values)
num_junctions = 3
length_junctions = [200, 300, 250]  # Length of each junction (in meters)
distance_between_junctions = [100, 150]  # Distance between each junction (in meters)
traffic_density = 70  # Traffic density as a percentage
average_speed = 30  # Average speed of vehicles (in km/h)

In [119]:
# Set up DEAP for the genetic algorithm

# Step 1: Create Fitness and Individual classes
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize the fitness function (traffic delay)
creator.create("Individual", list, fitness=creator.FitnessMin)

In [120]:
# Step 2: Define the individual creation function
toolbox = base.Toolbox()
toolbox.register("attr_green", random.randint, GREEN_MIN, GREEN_MAX)
toolbox.register("attr_yellow", random.randint, YELLOW_MIN, YELLOW_MAX)
toolbox.register("attr_red", random.randint, RED_MIN, RED_MAX)

In [121]:
# Each individual is represented by a tuple of (green, yellow, red) durations
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_green, toolbox.attr_yellow, toolbox.attr_red), n=num_junctions)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [122]:
# Step 3: Define the fitness function
def evaluate(individual):
    """
    Fitness function that calculates the traffic flow efficiency as throughput minus wait time.
    This function considers traffic flowing from North-South and East-West directions.
    """
    total_throughput = 0  # Total vehicles that passed through the junctions
    total_wait_time = 0    # Total waiting time for vehicles

    for i in range(num_junctions):
        # Extract the timings for the current junction
        green_time = individual[i * 3]
        yellow_time = individual[i * 3 + 1]
        red_time = individual[i * 3 + 2]

        # Validate the timings
        if not (GREEN_MIN <= green_time <= GREEN_MAX):
            return float('inf'),  # Return an infeasible solution if green time is out of bounds
        if not (YELLOW_MIN <= yellow_time <= YELLOW_MAX):
            return float('inf'),  # Return an infeasible solution if yellow time is out of bounds
        if not (RED_MIN <= red_time <= RED_MAX):
            return float('inf'),  # Return an infeasible solution if red time is out of bounds

        # Calculate throughput for the green phase
        vehicles_passed = (green_time * average_speed / 3.6) / length_junctions[i]  # Number of vehicles per green time
        total_throughput += max(vehicles_passed, 0)

        # Calculate waiting time
        # Assume a constant number of vehicles waiting during red
        # We can use traffic_density as a proxy for the number of waiting vehicles
        # Vehicles waiting during red and yellow times
        waiting_vehicles = traffic_density / 100 * (length_junctions[i] / average_speed) * (red_time + yellow_time)
        total_wait_time += max(waiting_vehicles, 0)

    # The fitness is defined as total throughput minus total wait time (maximize throughput and minimize wait)
    fitness = total_throughput - total_wait_time
    return fitness,

toolbox.register("evaluate", evaluate)

In [123]:
# Step 4: Register GA operations (selection, crossover, mutation)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutUniformInt, low=[GREEN_MIN, YELLOW_MIN, RED_MIN]*num_junctions, 
                 up=[GREEN_MAX, YELLOW_MAX, RED_MAX]*num_junctions, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [124]:
# Step 5: Run the GA
def run_ga():
    random.seed(42)
    population = toolbox.population(n=50)
    generations = 100

    # Run the genetic algorithm
    result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=generations, verbose=False)
    
    # Extract the best individual
    best_individual = tools.selBest(population, k=1)[0]
    best_green_yellow_red = [(best_individual[i*3], best_individual[i*3+1], best_individual[i*3+2]) for i in range(num_junctions)]
    
    print("Optimized green, yellow, and red durations per junction:")
    for j, durations in enumerate(best_green_yellow_red):
        print(f"Junction {j+1} - Green: {durations[0]} sec, Yellow: {durations[1]} sec, Red: {durations[2]} sec")

In [125]:
run_ga()

Optimized green, yellow, and red durations per junction:
Junction 1 - Green: 10.68384587142115 sec, Yellow: 9.95796054524655 sec, Red: 59.9791287535454 sec
Junction 2 - Green: 10.714413413393759 sec, Yellow: 9.999455800133875 sec, Red: 59.30559365790809 sec
Junction 3 - Green: 12.00150267755388 sec, Yellow: 9.964648695372041 sec, Red: 56.94248195347669 sec
